In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data = np.load('Subchannel/results/test_results.npz')

# branch inputs
input0 = data['branch1']
input1 = data['branch2'][:, 0]
input2 = data['branch2'][:, 1]

# predictions
pred = data['predictions']
# ground truth
gt = data['targets']

# print shapes
print(f'input0 shape: {input0.shape}')
print(f'input1 shape: {input1.shape}')
print(f'input2 shape: {input2.shape}')
print(f'pred shape: {pred.shape}')
print(f'gt shape: {gt.shape}')


# load mesh points
mesh_points = np.load('Subchannel/data/share/trunk_input.npz')['trunk']
x = mesh_points[:, 0]
y = mesh_points[:, 1]

input0 shape: (1000, 100)
input1 shape: (1000,)
input2 shape: (1000,)
pred shape: (1000, 1733, 3)
gt shape: (1000, 1733, 3)


In [7]:
# compute relative l2 error for each channel [# samples, # mesh points, # channels]
def rel_l2_error(pred, gt):
    # compute relative l2 error
    err = np.linalg.norm(pred - gt, axis=-1) / np.linalg.norm(gt, axis=-1)
    return err
# compute relative l2 error for each channel
err0 = rel_l2_error(pred[:, :, 0], gt[:, :, 0])
err1 = rel_l2_error(pred[:, :, 1], gt[:, :, 1])
err2 = rel_l2_error(pred[:, :, 2], gt[:, :, 2])

# compute mean relative l2 error for each channel
mean_err0 = np.mean(err0, axis=0)
mean_err1 = np.mean(err1, axis=0)
mean_err2 = np.mean(err2, axis=0)

# compute mean relative l2 error for each channel
mean_err0 = np.mean(mean_err0) * 100
mean_err1 = np.mean(mean_err1) * 100
mean_err2 = np.mean(mean_err2) * 100


# compute standard deviation of relative l2 error for each channel
std_err0 = np.std(err0, axis=0) * 100
std_err1 = np.std(err1, axis=0) * 100
std_err2 = np.std(err2, axis=0) * 100

# print mean and std of relative l2 error for each channel
print(f'Mean relative L2 error for channel 0: {mean_err0:.4f} +/- {std_err0:.4f}')
print(f'Mean relative L2 error for channel 1: {mean_err1:.4f} +/- {std_err1:.4f}')
print(f'Mean relative L2 error for channel 2: {mean_err2:.4f} +/- {std_err2:.4f}')

Mean relative L2 error for channel 0: 2.2111 +/- 0.0230
Mean relative L2 error for channel 1: 0.2682 +/- 0.1165
Mean relative L2 error for channel 2: 4.1496 +/- 0.0377


In [106]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation, PillowWriter

# Triangulation
tri = mtri.Triangulation(x, y)

# Set up figure
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
plt.subplots_adjust(hspace=0.4, wspace=0.6)

# Geometry parameters
pitch = 12.6e-3
radius = 9.5e-3 / 2
circle_centers = [
    (-pitch/2, pitch/2),
    (pitch/2, pitch/2),
    (-pitch/2, -pitch/2),
    (pitch/2, -pitch/2)
]

full_extent = 0.0063
x_min_zoom, x_max_zoom = 0, 0.0063
y_min_zoom, y_max_zoom = -0.0063, 0

# Channel names
channel_name = ['Velocity [m/s]', 'Temperature [K]', r'TKE $\rm [m^2/s^2]$']

# Store contour handles
contour_full = []
contour_zoom = []
cbars_full = []
cbars_zoom = []

# --- Initialization ---
for i in range(3):
    # Top row: full view
    cf_full = ax[0, i].tricontourf(tri, pred[0, :, i], levels=150, cmap='jet')
    contour_full.append(cf_full)
    cbar_full = fig.colorbar(cf_full, ax=ax[0, i], fraction=0.046, pad=0.04)
    cbar_full.set_label(channel_name[i], fontsize=16)
    cbars_full.append(cbar_full)

    ax[0, i].set_xlim(-full_extent, full_extent)
    ax[0, i].set_ylim(-full_extent, full_extent)
    ax[0, i].set_title('Full View', fontsize=14)
    ax[0, i].set_aspect('equal')

    # Add white circles
    for xc, yc in circle_centers:
        ax[0, i].add_patch(plt.Circle((xc, yc), radius, color='white', zorder=5))

    # Add zoom rectangle
    zoom_rect = patches.Rectangle(
        (x_min_zoom, y_min_zoom),
        x_max_zoom - x_min_zoom,
        y_max_zoom - y_min_zoom,
        linewidth=2,
        edgecolor='red',
        linestyle='--',
        facecolor='none',
        zorder=6
    )
    ax[0, i].add_patch(zoom_rect)

    # Bottom row: zoomed-in view
    cf_zoom = ax[1, i].tricontourf(tri, pred[0, :, i], levels=150, cmap='jet')
    contour_zoom.append(cf_zoom)
    cbar_zoom = fig.colorbar(cf_zoom, ax=ax[1, i], fraction=0.046, pad=0.04)
    cbar_zoom.set_label(channel_name[i], fontsize=16)
    cbars_zoom.append(cbar_zoom)

    ax[1, i].set_xlim(x_min_zoom, x_max_zoom)
    ax[1, i].set_ylim(y_min_zoom, y_max_zoom)
    ax[1, i].set_title('Zoomed-In', fontsize=14)
    ax[1, i].set_aspect('equal')

    for xc, yc in circle_centers:
        ax[1, i].add_patch(plt.Circle((xc, yc), radius, color='white', zorder=5))

plt.tight_layout()

# --- Update Function ---
def update(frame_idx):
    print(f"Rendering frame {frame_idx}", end='\r')
    for i in range(3):
        # Remove previous contours
        for coll in contour_full[i].collections:
            coll.remove()
        for coll in contour_zoom[i].collections:
            coll.remove()

        # Frame-specific min/max
        frame_min = pred[frame_idx, :, i].min()
        frame_max = pred[frame_idx, :, i].max()

        # Redraw
        contour_full[i] = ax[0, i].tricontourf(
            tri, pred[frame_idx, :, i], levels=150, cmap='jet',
            vmin=frame_min, vmax=frame_max
        )
        contour_zoom[i] = ax[1, i].tricontourf(
            tri, pred[frame_idx, :, i], levels=150, cmap='jet',
            vmin=frame_min, vmax=frame_max
        )

        # Update titles
        ax[0, i].set_title(f'Full View (Frame {frame_idx})', fontsize=14)
        ax[1, i].set_title(f'Zoomed-In (Frame {frame_idx})', fontsize=14)

    return contour_full + contour_zoom

# --- Create Animation ---
ani = FuncAnimation(fig, update, frames=np.arange(0, 150), blit=False)

# --- Save Animation ---
writer = PillowWriter(fps=2)
ani.save('visualization.gif', writer=writer)

plt.close()

print("\nAnimation saved successfully!")


/var/folders/m2/q04dcnhs4yj_6ml80j57mmk80000gp/T/ipykernel_42249/365020677.py:90: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed in 3.10.
  for coll in contour_full[i].collections:
/var/folders/m2/q04dcnhs4yj_6ml80j57mmk80000gp/T/ipykernel_42249/365020677.py:92: MatplotlibDeprecationWarning: The collections attribute was deprecated in Matplotlib 3.8 and will be removed in 3.10.
  for coll in contour_zoom[i].collections:


Rendering frame 149
Animation saved successfully!
